# Aim

To perform analysis on 253 samples downloaded from Israeli dataset. First part is to merge all CpGs into one dataframe. For this we will use data.table library for fast reading and writing. Data should already be divided chromosome wise. This has been done on bash using awk

## Prepare the chromsome wise matrix

For each chromosome beta file was converted to bed format using wgbstools. More infromation about beta format and wgbstools check [here](https://github.com/nloyfer/wgbs_tools/blob/master/docs/beta_format.md).

**beta file**: beta file is the simplest representation of methylation data. It is a binary file columns

-   meth: the number of times the i'th site (CpGi) site is observed in a methylated state.

-   coverage: the total number of times i'th site (CpGi) is observed. #coverage==0 is equivalent to a missing value (NaN). CpGi beta value is obtained by dividing #meth /#coverage.

Read all the files and then use divide for each chromosome


```{bash,eval=FALSE}
cd /jmsh/external/nihit/Israeli_methylation_dataset/matrices
```


Following command will go through each file and divide based on column1 which is chromosome and for each file it extracts the file name and save it in the folder chromosome/samplename_chromosome.bed

Only first three columns are saved \# change to print \$5 incase you need coverage and \$4 if you need methylation (Keep in mind here 5th column is coverage and 4th column is methyated reads only because we get it from beta files)


```{bash,eval=FALSE}
# the following command will go through each file and divide based on column1 which is chromosome and for each file it extracts the file name and save it in the folder ${chromosome}/${samplename}_{chromosome}.bed 
# only first three columns are saved # change to print $5 incase you need coverage and $4 if you need methylation (Keep in mind here 5th column is coverage and 4th column is methyated reads only because we get it from beta files)

for files in $(ls ../bed/*bed |grep -v counts )
do 
  fname=$(basename $files .hg38.bed)
  awk -v a="$fname" -vOFS="\t" '{print $1,$2,$3 > $1"/"a"_"$1".bed"}' $files
done
```


Above command was cancelled as I could think of efficient way to get all coordinates across files. I simply need to extract chromosome wise second column from each file and this can be used as the coordinate \# Adapted command only print column 2


```{bash,eval=FALSE}
# the following command will go through each file and divide based on column1 which is chromosome and for each file it extracts the file name and save it in the folder ${chromosome}/${samplename}_{chromosome}.bed 
# only first three columns are saved # change to print $5 incase you need coverage and $4 if you need methylation (Keep in mind here 5th column is coverage and 4th column is methyated reads only because we get it from beta files)

for files in $(ls ../bed/*bed |grep -v counts )
do 
  fname=$(basename $files .hg38.bed)
  awk -v a="$fname" -vOFS="\t" '{print $2 > $1"/"a"_"$1".bed"}' $files
done
```


## Merge all CpG coordinates

Here I am checking and extracting all the CpG coordinates covered across all samples. Finally for each chromosome a sorted bed file is created later this would be use to generate chromosome wise matrix.


```{bash,eval=FALSE}
# Working directory 
cd /jmsh/external/nihit/Israeli_methylation_dataset/matrices
# cat over all chromosome folder, sort and use only uniq to get all coordinates of CpGs across samples 
for chromosomes in $(ls c* -d )
do 
  echo $chromosomes
  cat ${chromosomes}/*txt \
  |sort \
  |uniq \
  > ${chromosomes}.txt
done

# sort the coordinates for each chromosome in increasing bp
for files in chr*txt
do 
  fname=$(basename $files .txt)
  sort -k1,1n $files \
  |awk -v a="$fname" -vOFS="\t" '{print a,$1,$1+2 }' \
  > ${fname}_sorted.bed
done
```


I decided to use bash and bedtools intersect for generating chromosome wise matrices for coverage and methylation as the matrix would be quite large to handle in R since even readr was quite slow.


```{bash,eval=FALSE}
# change to the files directory
cd /jmsh/external/nihit/Israeli_methylation_dataset/matrices

# activating the conda environment. Later would be needed would for bedtools intersect
source /jmsh/projects/conda/minconda/bin/activate /jmsh/projects/conda/minconda/envs/bins14_core/

for files in $(ls ../bed/*.hg38.bed)
do 
  i=$(expr $i + 1)
  fname=$(basename $files .hg38.bed)
  echo -e "Processing sample number: ${i} - ${fname}"
  awk -v a="$fname" -vOFS="\t" '{print $1,$2,$3,$5,$4 > $1"/"a"_"$1".temp.bed"}' $files
done
```

```{bash,eval=FALSE}

## Merge all samples chromosome wise

# load data.table 
library(readr)
library(dplyr)

# set file paths
setwd("/jmsh/projects/researchers/bins14/AG_Israel/Analysis/")
folder="/jmsh/external/nihit/Israeli_methylation_dataset/bed/"
chromosome="Y"

coordinate_file <- read_tsv(file = "/jmsh/external/nihit/Israeli_methylation_dataset/matrices/chrY_sorted.bed",
                            col_names = FALSE)


list_of_files <- list.files(folder, full.names = TRUE,pattern=".hg38.bed$")

# reading all files 
files = lapply(list_of_files[1:2],function(x){
  sample_data <- read_tsv(x,col_names = FALSE,show_col_types = FALSE)
  sample_data <- sample_data[c(sample_data$X1==paste("chr",chromosome,sep = "")),]
  sample_data <- subset(sample_data,select=c("X2")) 

  return(sample_data)
  })

merged_df <- as.data.frame(do.call(cbind,files))

```